<a href="https://colab.research.google.com/github/cconsta1/age-estimation-notebook/blob/main/age_estimation_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install necessary libraries**

In [ ]:
!pip install scikit-optimize git+https://github.com/hyperopt/hyperopt-sklearn.git  

# **Import necessary libraries**

In [ ]:
# Google colab

from google.colab import data_table
from google.colab import files

data_table.enable_dataframe_formatter()

# hyperopt

import hyperopt
from hyperopt.pyll import as_apply
from hyperopt import tpe, hp
import hpsklearn
from hpsklearn import HyperoptEstimator, random_forest_classifier, extra_trees_classifier, bagging_classifier, \
  ada_boost_classifier, gradient_boosting_classifier, hist_gradient_boosting_classifier, bernoulli_nb, \
  categorical_nb, complement_nb, gaussian_nb, multinomial_nb, sgd_classifier, sgd_one_class_svm, \
  ridge_classifier, ridge_classifier_cv, passive_aggressive_classifier, perceptron, \
  dummy_classifier, gaussian_process_classifier, mlp_classifier, linear_svc, nu_svc, svc, \
  decision_tree_classifier, extra_tree_classifier, label_propagation, label_spreading, elliptic_envelope, \
  linear_discriminant_analysis, quadratic_discriminant_analysis, bayesian_gaussian_mixture, gaussian_mixture, \
  k_neighbors_classifier, radius_neighbors_classifier, nearest_centroid, \
  xgboost_classification, lightgbm_classification, mlp_regressor

from hpsklearn.components import all_classifiers, all_preprocessing, any_classifier, any_preprocessing, \
any_regressor, all_regressors

# Hyperparameter optimization

import skopt
from skopt import BayesSearchCV

# system

import os
import io
import re

# data analysis and plotting

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from math import sqrt

from scipy.stats import zscore, shapiro
from random import randint

# data processing and model validation

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, \
accuracy_score, classification_report, log_loss, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold, KFold, \
LeaveOneOut, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold, StratifiedKFold

# classification libraries

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, Matern, RationalQuadratic
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, \
ExtraTreesRegressor, ExtraTreesClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier, plot_importance

#import lightgbm as lgb

# Importing imputation libs

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

# Missing data models

from itertools import combinations
from joblib import parallel_backend

# Export models into pickle
import pickle

# Tensorflow 

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
# import scikeras
#from scikeras.wrappers import KerasClassifier, KerasRegressor

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor


# Various parameter settings

#%matplotlib inline


# To change scientific numbers to float
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
#sns.set(rc={'figure.figsize':(12,10)})

# import sys

# Displaying all the rows/columns in a data set (the default option is not to show them)

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# **Importing and preparing the data for the analysis**

In [ ]:
uploaded = files.upload()

In [ ]:
raw_data = pd.read_csv(io.BytesIO(uploaded['age_dataset.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
raw_data.head()

In [ ]:
df = raw_data.iloc[:,[2, 4, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 38, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 3]]

In [ ]:
df = pd.DataFrame(df.values[3:], columns=df.iloc[2])

df = df.astype(int)

df

In [ ]:
df.describe()

In [ ]:
# Add a new target vector called age groups

df['Age_groups'] = pd.cut(df['Age'], bins=[10,35,50,100], labels=False)

df = df.astype(int)

df

In [ ]:
# View the data as a table

data_table.DataTable(df, include_index=False, num_rows_per_page=10, max_columns=40)

# **Variables dictionary**

In [ ]:
independent_variables_sets = {
    "Suchey Brooks 1990": [
        'Right Phase Suchey'
        ],
    "Meindl and Lovejoy": [
        'Right 1-midlamdoid',
        '2-lambda', 
        '3-obelion', 
        '4-anterior sagital',
        '5-bregma', 
        'Right 6-midcoronal', 
        'Right 7-pterion',
        'Right 8-sphenofrontal', 
        'Right 9-inferior sphenotemporal', 
        'Right 10-superior sphenotemporal'
        ],
    "Lovejoy et al": [
        "Right Phase"
    ],
    "Buckberry and Chamberlain": [
        'Right Transverse organization',
        'Right Surface texture',
        'Right Microposity', 
        'Right Macroporositty', 
        'Right Apical changes'
        ],
    "Suchey Brooks 1990 and Lovejoy et al": [
        'Right Phase Suchey',
        'Right Phase' 
    ],
    "Suchey Brooks 1990 and Buckberry Chamberlain": [
        'Right Transverse organization',
        'Right Surface texture',
        'Right Microposity', 
        'Right Macroporositty', 
        'Right Apical changes',
        'Right Phase Suchey'
    ],
    "All": [
        'Right Phase Suchey',
        'Right 1-midlamdoid',
        '2-lambda', 
        '3-obelion', 
        '4-anterior sagital',
        '5-bregma', 
        'Right 6-midcoronal', 
        'Right 7-pterion',
        'Right 8-sphenofrontal', 
        'Right 9-inferior sphenotemporal', 
        'Right 10-superior sphenotemporal',
        "Right Phase",
        'Right Transverse organization',
        'Right Surface texture',
        'Right Microposity', 
        'Right Macroporositty', 
        'Right Apical changes'
    ]
} 


# **Classification (sklearn)** 

# **Sex specific**

In [ ]:
# Note: Change df[df["sex"]==2] to df[df["sex"]==1] for male

dff = df[df["sex"]==2]
y = dff['Age_groups'].values

for key, value in independent_variables_sets.items():
  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25, stratify=y)

  model = HyperoptEstimator(classifier=any_classifier('classifier'), preprocessing=any_preprocessing('preprocessing'), \
                          algo=tpe.suggest, max_evals=100, trial_timeout=100, continuous_loss_fn=False)

  model.fit(X_train, y_train)

  # summarize performance
  acc = model.score(X_test, y_test)
  cnfm = confusion_matrix(y_test, model.predict(X_test))

  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # Run a LOOCV for model

  result_loocv = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=LeaveOneOut(), error_score='raise')

  # save information and model

  filename = ''.join(['classification_right_women_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['classification_right_women_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("Accuracy: {}\n".format(acc))
    infofile.write("Confusion matrix: \n{}\n".format(cnfm))
    infofile.write("LOOCV accuracy: {}\n".format(result_loocv.mean()))


In [ ]:
with open('classification_right_women_Suchey_Brooks_1990.txt', 'r') as f:
    print(f.read())

# **Sex blind**

In [ ]:
dff = df 
y = dff['Age_groups'].values

for key, value in independent_variables_sets.items():
  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25, stratify=y)

  model = HyperoptEstimator(classifier=any_classifier('classifier'), preprocessing=any_preprocessing('pre'), \
                          algo=tpe.suggest, max_evals=100, trial_timeout=100)

  model.fit(X_train, y_train)

  # summarize performance
  acc = model.score(X_test, y_test)
  cnfm = confusion_matrix(y_test, model.predict(X_test))

  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # Run a LOOCV for model

  result_loocv = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=LeaveOneOut(), error_score='raise')

  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # save information and model

  filename = ''.join(['classification_right_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['classification_right_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("Accuracy: {}\n".format(acc))
    infofile.write("Confusion matrix: \n{}\n".format(cnfm))
    infofile.write("LOOCV accuracy: {}\n".format(result_loocv.mean()))



In [ ]:
with open('classification_right_Suchey_Brooks_1990_and_Buckberry_Chamberlain.txt', 'r') as f:
    print(f.read())

In [ ]:
with open('classification_right_Suchey_Brooks_1990_and_Buckberry_Chamberlain.dat', 'rb') as f:
  model = pickle.load(f)
  y_pred = model.predict_proba(X_test)
  print(y_pred)

# **Regression (sklearn)**

# **Sex specific**

In [ ]:
dff = df[df["sex"]==2]
y = dff['Age'].values

for key, value in independent_variables_sets.items():

  print("key: ", key)
  print("value: ", value)

  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)

  model = HyperoptEstimator(regressor=any_regressor('regressor'), preprocessing=any_preprocessing('pre'), \
                          algo=tpe.suggest, max_evals=100, loss_fn=mean_absolute_error, \
                          trial_timeout=100,continuous_loss_fn=False, verbose=True)

  model.fit(X_train, y_train)

  # summarize performance

  #predictions
  y_pred = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  # metrics

  #acc = model.score(X_test, y_test) # R^2 on test set
  r2_test = r2_score(y_test, y_pred) # R^2 on test set using sklearn (same as above)
  r2_train = r2_score(y_train, y_pred_train) # R^2 on training set
  rmse = mean_squared_error(y_pred, y_test, squared=False) # root mean squared error
  mae = mean_absolute_error(y_pred, y_test) # mean absolute error

  # train model again on the full dataset using best estimator and hyperparamaters

  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  plt.plot(pipe.predict(X_test),'ro')
  plt.plot(y_test,'b*')
  plt.title(key)
  plt.show()
  plt.clf()

  # save model and model information  

  filename = ''.join(['regression_right_women_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['regression_right_women_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("R **2 = {} (test), R**2 = {} (train)\n".format(r2_test, r2_train))
    infofile.write("RMSE = {}\n".format(rmse))
    infofile.write("MAE = {}\n".format(mae))



In [ ]:
with open('regression_right_women_All.txt', 'r') as f:
    print(f.read())

In [ ]:
with open('regression_right_women_All.dat', 'rb') as f:
  pickle.load(f).predict(X_test)

In [ ]:
with open('regression_right_women_All.dat', 'rb') as f:
  plt.plot(pickle.load().predict(X_test),'ro')
  plt.plot(y_test,'b*')

# **Calculate Bias-Inaccuracy**

In [ ]:
import pandas as pd

dff = df[df['sex']==2]
y = dff['Age'].values

output_list = []

for key, value in independent_variables_sets.items():

    print("key: ", key)
    #print("value: ", value)

    X = dff[value].values

    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)

    # model

    filename = ''.join(['regression_right_women_', key.replace(" ","_"), ".dat"])
  
    with open(filename, "rb") as modelfile:
        model = pickle.load(modelfile)

        #predictions
        y_pred = model.predict(X)
        #print((y_pred-y))
        bias = np.sum((y_pred-y))/len(X)
        inaccuracy = np.sum(np.absolute(y_pred-y))/len(X)
        
        output_dict = {'key': key, 'bias': bias, 'inaccuracy': inaccuracy}
        output_list.append(output_dict)

df_bias_inaccuracy = pd.DataFrame(output_list)

df_bias_inaccuracy


# **Sex blind**

In [ ]:
dff = df 
y = dff['Age'].values

for key, value in independent_variables_sets.items():

  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)

  model = HyperoptEstimator(
      regressor=any_regressor('regressor'), 
      preprocessing=any_preprocessing('pre'), 
      algo=tpe.suggest, 
      max_evals=100, 
      trial_timeout=100
      )

  model.fit(X_train, y_train)

  # summarize performance

  #predictions
  y_pred = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  # metrics

  #acc = model.score(X_test, y_test) # R^2 on test set
  r2_test = r2_score(y_test, y_pred) # R^2 on test set using sklearn (same as above)
  r2_train = r2_score(y_train, y_pred_train) # R^2 on training set
  rmse = mean_squared_error(y_pred, y_test, squared=False) # root mean squared error
  mae = mean_absolute_error(y_pred, y_test) # mean absolute error

  # train model again on the full dataset using best estimator and hyperparamaters

  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # plot results

  plt.plot(pipe.predict(X_test),'ro')
  plt.plot(y_test,'b*')
  plt.title(key)
  plt.show()
  plt.clf()

  # save model and model information  

  filename = ''.join(['regression_right_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['regression_right_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("R **2 = {} (test), R**2 = {} (train)\n".format(r2_test, r2_train))
    infofile.write("RMSE = {}\n".format(rmse))
    infofile.write("MAE = {}\n".format(mae))


# **Neural Networks**

# **Classification (MLPClassifier)**

# **Sex specific**

In [ ]:
dff = df[df["sex"]==2]
y = dff['Age_groups'].values

for key, value in independent_variables_sets.items():
  
  X = dff[value].values
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25, stratify=y)

  model = HyperoptEstimator(classifier=mlp_classifier("mlp_classifier", \
                          hidden_layer_sizes=hp.choice('hidden_layer_sizes', [(10,), (20,), (100,), (10, 10), (100, 100)])), \
                          preprocessing=any_preprocessing("preprocessing"), \
                          algo=tpe.suggest, max_evals=100, trial_timeout=100)

  model.fit(X_train, y_train)

  # summarize performance
  acc = model.score(X_test, y_test)
  cnfm = confusion_matrix(y_test, model.predict(X_test))

  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # Run a 20-fold CV model
  seed = 7
  np.random.seed(seed)

  kFold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

  result_loocv = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=kFold, error_score='raise')

  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # save information and model

  filename = ''.join(['ann_classification_right_women_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['ann_classification_right_women_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("Accuracy: {}\n".format(acc))
    infofile.write("Confusion matrix: \n{}\n".format(cnfm))
    infofile.write("20-fold accuracy: {}\n".format(result_loocv.mean()))



# **Sex blind**

In [ ]:
dff = df 
y = dff['Age_groups'].values

for key, value in independent_variables_sets.items():
  
  X = dff[value].values
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25, stratify=y)

  model = HyperoptEstimator(classifier=mlp_classifier("mlp_classifier", \
                          hidden_layer_sizes=hp.choice('hidden_layer_sizes', [(10,), (20,), (100,), (10, 10), (100, 100)])), \
                          preprocessing=any_preprocessing("preprocessing"), \
                          algo=tpe.suggest, max_evals=100, trial_timeout=100)

  model.fit(X_train, y_train)

  # summarize performance
  acc = model.score(X_test, y_test)
  cnfm = confusion_matrix(y_test, model.predict(X_test))

  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # Run a 20-fold CV model
  seed = 7
  np.random.seed(seed)

  kFold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

  result_loocv = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=kFold, error_score='raise')

  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # save information and model

  filename = ''.join(['ann_classification_right_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['ann_classification_right_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("Accuracy: {}\n".format(acc))
    infofile.write("Confusion matrix: \n{}\n".format(cnfm))
    infofile.write("20-fold accuracy: {}\n".format(result_loocv.mean()))



In [ ]:
with open('ann_classification_right_Suchey_Brooks_1990.txt', 'r') as f:
  print(f.read())

# **Regression (MLPRegressor)**

# **Sex specific**

In [ ]:
dff = df[df["sex"]==2]
y = dff['Age'].values 

for key, value in independent_variables_sets.items():
  
  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)
 
  layers = hp.choice('layers', np.arange(10, 15))
  solver = hp.choice('sol', ['lbfgs', "lbfgs", "sgd", "adam"])
  iter = hp.choice('iter', [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000])
  alpha = hp.choice('alph', 10.0 ** -np.arange(1, 10))
  random_state=hp.choice('rand', [0,1,2,3,4,5,6,7,8,9])

  space = as_apply({
      "classifier": None,
      "regressor": mlp_regressor('mlp', 
                                hidden_layer_sizes = layers, 
                                solver=solver,
                                max_iter=iter,
                                alpha=alpha,
                                random_state=random_state,
                                batch_size="auto",
                                learning_rate="constant",
                                learning_rate_init=0.001,
                                power_t=0.5,
                                shuffle=True,
                                tol=1e-4,
                                verbose=False,
                                warm_start=False,
                                momentum=0.9,
                                nesterovs_momentum=True,
                                early_stopping=False,
                                validation_fraction=0.1,
                                beta_1=0.9,
                                beta_2=0.999,
                                epsilon=1e-8,
                                n_iter_no_change=10,
                                max_fun=15000
                                ),
      "preprocessing": any_preprocessing("pre"),
      "ex_preprocs": []})
  

  model = HyperoptEstimator(space=space, algo=tpe.suggest, max_evals=100, trial_timeout=100, verbose=True, loss_fn=mean_absolute_error)

  model.fit(X_train, y_train)

  # summarize performance

  #predictions
  y_pred = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  # metrics

  #acc = model.score(X_test, y_test) # R^2 on test set
  r2_test = r2_score(y_test, y_pred) # R^2 on test set using sklearn (same as above)
  r2_train = r2_score(y_train, y_pred_train) # R^2 on training set
  rmse = mean_squared_error(y_pred, y_test, squared=False) # root mean squared error
  mae = mean_absolute_error(y_pred, y_test) # mean absolute error


  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # plot results

  plt.plot(pipe.predict(X_test),'ro')
  plt.plot(y_test,'b*')
  plt.title(key)
  plt.show()
  plt.clf()

  # save information and model

  filename = ''.join(['ann_regression_right_women_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['ann_regression_right_women_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    #infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("R **2 = {} (test), R**2 = {} (train)\n".format(r2_test, r2_train))
    infofile.write("RMSE = {}\n".format(rmse))
    infofile.write("MAE = {}\n".format(mae))



# **Sex blind**

In [ ]:
dff = df
y = dff['Age'].values 

for key, value in independent_variables_sets.items():
  
  X = dff[value].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)


  # parameters = {
  #     'solver': ['lbfgs', "lbfgs", "sgd", "adam"], 
  #     'max_iter': [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], 
  #     'alpha': 10.0 ** -np.arange(1, 10), 
  #     'hidden_layer_sizes':np.arange(10, 15), 
  #     'random_state':[0,1,2,3,4,5,6,7,8,9]
  #     }

 
  layers = hp.choice('layers', np.arange(10, 15))
  solver = hp.choice('sol', ['lbfgs', "lbfgs", "sgd", "adam"])
  iter = hp.choice('iter', [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000])
  alpha = hp.choice('alph', 10.0 ** -np.arange(1, 10))
  random_state=hp.choice('rand', [0,1,2,3,4,5,6,7,8,9])

  space = as_apply({
      "classifier": None,
      "regressor": mlp_regressor('mlp', 
                                hidden_layer_sizes = layers, 
                                solver=solver,
                                max_iter=iter,
                                alpha=alpha,
                                random_state=random_state,
                                batch_size="auto",
                                learning_rate="constant",
                                learning_rate_init=0.001,
                                power_t=0.5,
                                shuffle=True,
                                tol=1e-4,
                                verbose=False,
                                warm_start=False,
                                momentum=0.9,
                                nesterovs_momentum=True,
                                early_stopping=False,
                                validation_fraction=0.1,
                                beta_1=0.9,
                                beta_2=0.999,
                                epsilon=1e-8,
                                n_iter_no_change=10,
                                max_fun=15000
                                ),
      "preprocessing": any_preprocessing("pre"),
      "ex_preprocs": []})
  

  model = HyperoptEstimator(space=space, algo=tpe.suggest, max_evals=100, trial_timeout=100, verbose=True, loss_fn=mean_absolute_error)
  

  model.fit(X_train, y_train)

  #print(model.best_params_)
  
  #model.fit(X_train, y_train)

  # summarize performance

  #predictions
  y_pred = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  # metrics

  #acc = model.score(X_test, y_test) # R^2 on test set
  r2_test = r2_score(y_test, y_pred) # R^2 on test set using sklearn (same as above)
  r2_train = r2_score(y_train, y_pred_train) # R^2 on training set
  rmse = mean_squared_error(y_pred, y_test, squared=False) # root mean squared error
  mae = mean_absolute_error(y_pred, y_test) # mean absolute error


  # retrain on full dataset using best parameters
  pipe = Pipeline([('scaler', model.best_model()['preprocs'][0] ), ('clf', model.best_model()['learner'] )])
  pipe.fit(X, y)

  # plot results

  plt.plot(pipe.predict(X_test),'ro')
  plt.plot(y_test,'b*')
  plt.title(key)
  plt.show()
  plt.clf()

  # save information and model

  filename = ''.join(['ann_regression_right_', key.replace(" ","_"), ".dat"])
  infofilename = ''.join(['ann_regression_right_', key.replace(" ","_"), ".txt"])

  with open(filename, "wb") as modelfile:
    pickle.dump(pipe, modelfile)

  with open(infofilename, "w") as infofile:
    infofile.write("---------------------------------\n")
    infofile.write(key + '\n')
    infofile.write("Dataset size: {} {}\n".format(len(X), len(y)))
    #infofile.write("Best classifier: {}\n".format(model.best_model()))
    infofile.write("R **2 = {} (test), R**2 = {} (train)\n".format(r2_test, r2_train))
    infofile.write("RMSE = {}\n".format(rmse))
    infofile.write("MAE = {}\n".format(mae))



# **Calculate bias**

In [ ]:
import pandas as pd

dff = df#[df['sex']==2]
y = dff['Age'].values

output_list = []

for key, value in independent_variables_sets.items():

    X = dff[value].values

    #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25)

    # model

    filename = ''.join(['ann_regression_right_', key.replace(" ","_"), ".dat"])
  
    with open(filename, "rb") as modelfile:
        model = pickle.load(modelfile)

        #predictions
        y_pred = model.predict(X)
        #print((y_pred-y))
        bias = np.sum((y_pred-y))/len(X)
        inaccuracy = np.sum(np.absolute(y_pred-y))/len(X)
        
        output_dict = {'key': key, 'bias': bias, 'inaccuracy': inaccuracy}
        output_list.append(output_dict)

df_bias_inaccuracy = pd.DataFrame(output_list)

df_bias_inaccuracy


# **Zip results and download**

In [ ]:
!zip -r /content/results.zip /content/*

In [ ]:
files.download("/content/results.zip")
